In [31]:
import jsonlines
import json
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import RFE
import numpy as np
from scipy.sparse import csr_matrix
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
import string
import re
from os.path import join
from random import shuffle, sample
from sklearn.utils import shuffle
warnings.filterwarnings("ignore")

In [3]:
#SIRVE PARA TOKENIZAR
def to_single_message_format(gamefile):
    messages = []
    with open(gamefile) as inh:
        for ln in inh:
            conversation = json.loads(ln)
            for msg, sender_label, receiver_label, score_delta \
                in zip(conversation['messages'],conversation['sender_labels'], \
                    conversation['receiver_labels'], conversation['game_score_delta']):
                messages.append({'message': msg, 'receiver_annotation': receiver_label,\
                    'sender_annotation':sender_label, 'score_delta': int(score_delta)})
    shuffle(messages)    
    return messages

def write_single_messages(messages, outfile):
    with open(outfile, "w") as outh:
        for msg in messages:
            outh.write(json.dumps(msg)+'\n')

if __name__ == '__main__':
    ROOT = 'data/'
    

In [34]:
#LECTURA DE DATOS
df_train = to_single_message_format(join(ROOT, 'train.jsonl'))
df_test = to_single_message_format(join(ROOT, 'test.jsonl'))
df_validation = to_single_message_format(join(ROOT, 'validation.jsonl'))
#TRANSFORMACION A DATAFRAME
df_train_copy = pd.DataFrame(df_train)
df_test_copy = pd.DataFrame(df_test)
df_validation_copy = pd.DataFrame(df_validation)
y_train = df_train_copy['sender_annotation']
print('Y_train',y_train.shape)

# Revisar la distribución de clases en el conjunto de entrenamiento
print("Distribución de clases en el conjunto de entrenamiento:")
print(df_train_copy['sender_annotation'].value_counts())

# Revisar la distribución de clases en el conjunto de validación
print("Distribución de clases en el conjunto de test:")
print(df_test_copy['sender_annotation'].value_counts())

# Revisar la distribución de clases en el conjunto de validación
print("Distribución de clases en el conjunto de validación:")
print(df_validation_copy['sender_annotation'].value_counts())


#EXISTE UN DESBALANCEO DE CLASES, POR LO QUE SE DEBE HACER UN SOBREMUESTREO DE LA CLASE MINORITARIA
# Separar los datos por clases
df_majority = df_train_copy[df_train_copy['sender_annotation'] == True]
df_minority = df_train_copy[df_train_copy['sender_annotation'] == False]
# Sobremuestrear la clase minoritaria
df_minority_oversampled = df_minority.sample(len(df_majority), replace=True)  # Replicar los datos de la clase minoritaria
# Combinar la clase mayoritaria con la clase minoritaria sobremuestreada
df_oversampled = pd.concat([df_majority, df_minority_oversampled])
# Barajar o sedordena los datos
df_oversampled = shuffle(df_oversampled) 

# Ahora puedes proceder a tokenizar y preparar estos datos para el entrenamiento como lo hiciste anteriormente
# Revisar la nueva distribución de clases en el conjunto de entrenamiento
print("Distribución de la clases en el nuevo conjunto de entrenamiento:")
print(df_oversampled['sender_annotation'].value_counts())

df_train_copy = df_oversampled
y_train = df_train_copy['sender_annotation']
print('Y_train nuevo',y_train.shape)


print("Dimensiones de df_train_copy:", df_train_copy.shape)
print("Dimensiones de df_test_copy:", df_test_copy.shape)
print("Dimensiones de df_validation_copy:", df_validation_copy.shape)
print("Dimensiones de y_train:", y_train.shape)
print("*******************************:")
print("Dimensiones de df_train:", df_train_copy.shape)


Y_train (13132,)
Distribución de clases en el conjunto de entrenamiento:
True     12541
False      591
Name: sender_annotation, dtype: int64
Distribución de clases en el conjunto de test:
True     2501
False     240
Name: sender_annotation, dtype: int64
Distribución de clases en el conjunto de validación:
True     1360
False      56
Name: sender_annotation, dtype: int64
Distribución de la clases en el nuevo conjunto de entrenamiento:
True     12541
False    12541
Name: sender_annotation, dtype: int64
Y_train nuevo (25082,)
Dimensiones de df_train_copy: (25082, 4)
Dimensiones de df_test_copy: (2741, 4)
Dimensiones de df_validation_copy: (1416, 4)
Dimensiones de y_train: (25082,)
*******************************:
Dimensiones de df_train: (25082, 4)


In [35]:
#df_train_copy[0]
#df_test_copy[0]
#df_train_copy
df_train[0]

{'message': 'Germany!\n\nJust the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. \n\nSo...do I suggest it?\n\nI’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.\n\nWhat is your appetite like for unusual and crazy?',
 'receiver_annotation': True,
 'sender_annotation': True,
 'score_delta': 0}

In [36]:
#CREACION DEL MODELO
from transformers import BertTokenizer
import torch

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_messages(messages, max_length):
    input_ids = []
    attention_masks = []

    for message in messages:
        encoded_dict = tokenizer.encode_plus(
            message,                      
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Concatenar los tensores
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Aplicar la función de codificación a tus mensajes
messages = df_train_copy['message'].tolist()
input_ids, attention_masks = encode_messages(messages, max_length=256)

# Comprobación de las dimensiones
print("Dimensiones de input_ids:", input_ids.shape)
print("Dimensiones de attention_masks:", attention_masks.shape)
print("Dimensiones de y_train_tensor:", y_train.shape)

Dimensiones de input_ids: torch.Size([25082, 256])
Dimensiones de attention_masks: torch.Size([25082, 256])
Dimensiones de y_train_tensor: (25082,)


In [38]:
print(type(df_train))  # Debería ser <class 'list'>
print(all(isinstance(item, str) for item in df_train))  # Todos los elementos deben ser cadenas de texto
df_train = [str(message) for message in df_train]  # Convertir todos los elementos a cadenas de texto
print(type(df_train))  # Debería ser <class 'list'>
print(all(isinstance(item, str) for item in df_train))  # Todos los elementos deben ser cadenas de texto
print(df_train[:5])  # Imprime los primeros 5 elementos para verificar

<class 'list'>
False
<class 'list'>
True
["{'message': 'Germany!\\n\\nJust the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. \\n\\nSo...do I suggest it?\\n\\nI’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.\\n\\nWhat is your appetite like for unusual and crazy?', 'receiver_annotation': True, 'sender_annotation': True, 'score_delta': 0}", '{\'message\': "You\'ve whet my appetite, Italy. What\'s the suggestion?", \'receiver_annotation\': True, \'sender_annotation\': True, \'score_delta\': 0}', "{'message': '👍', 'receiver_annotation': True, 'sender_annotation': True, 'score_delta': 0}", '{\'message\': "It seems like there are a lot of ways that could go wrong...I don\'t see why France would see you approaching/tak

In [39]:
y_train
# Si y_train es una serie de Pandas (por ejemplo, una columna de DataFrame)
y_train_tensor = torch.tensor(y_train.values).long()
y_train_tensor.shape

print("Dimensiones de input_ids:", input_ids.shape)
print("Dimensiones de attention_masks:", attention_masks.shape)
print("Dimensiones de y_train_tensor:", y_train_tensor.shape)

Dimensiones de input_ids: torch.Size([25082, 256])
Dimensiones de attention_masks: torch.Size([25082, 256])
Dimensiones de y_train_tensor: torch.Size([25082])


In [40]:
#TensorDataset
#es una clase de PyTorch que encapsula tensores en un conjunto de datos. 
#En tu caso, estos tensores serán los input_ids y attention_masks generados por el tokenizador,
#así como las etiquetas de tus datos (y_train).
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(input_ids, attention_masks, y_train_tensor)

In [41]:
#DataLoader
#DataLoader es una clase que proporciona un iterable sobre el conjunto de datos. Con DataLoader, puedes especificar 
#el tamaño del lote (batch size), si los datos deben ser mezclados, y otros parámetros que son útiles durante el entrenamiento.
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [42]:
print("Dimensiones de train_dataloader:", train_dataset)
print("Dimensiones de train_dataloader:", train_dataloader)


Dimensiones de train_dataloader: <torch.utils.data.dataset.TensorDataset object at 0x0000025DFFF52410>
Dimensiones de train_dataloader: <torch.utils.data.dataloader.DataLoader object at 0x0000025DFFD78190>


In [43]:
import torch

print(torch.cuda.is_available())  # Debería ser True si CUDA está disponible
print(torch.version.cuda)        # Debería mostrar la versión de CUDA
print(torch.cuda.get_device_name(0))  # Muestra el nombre de tu GPU CUDA


True
11.8
NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [44]:
#CONFIGURACION DEL MODELO BERT
import torch
print(torch.version.cuda)

11.8


In [45]:
import torch
#C:\Users\nobce\AppData\Local\Temp\cuda
if torch.cuda.is_available():
    print("CUDA disponible. Dispositivos CUDA:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible.")


CUDA disponible. Dispositivos CUDA: 1
Nombre del dispositivo CUDA: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [46]:
from transformers import BertForSequenceClassification, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el modelo preentrenado de BERT
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Usa la versión base de BERT
    num_labels = 2,     # Número de etiquetas de salida (por ejemplo, 2 para clasificación binaria)
    output_attentions = False, # Si el modelo debe retornar las atenciones
    output_hidden_states = False, # Si el modelo debe retornar todos los estados ocultos
)

# Mover el modelo al dispositivo adecuado (GPU o CPU)
model.to(device)  # Ejemplo: device puede ser 'cuda' para GPU o 'cpu' para CPU

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [47]:
for step, batch in enumerate(train_dataloader):
    # Mover el lote al dispositivo
    b_input_ids, b_input_mask, b_labels = batch
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)
    b_labels = b_labels.to(device)

    # Realizar pasos de entrenamiento...


In [48]:
# Definir el optimizador y la Función de Pérdida
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # tasa de aprendizaje
                  eps = 1e-8 # epsilon
                 )

# Definir la función de pérdida
from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss()

In [49]:
# Número de épocas de entrenamiento
epochs = 2

# Bucle para cada época
for epoch in range(epochs):
    
    # Entrenamiento
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        
        # Desempacar el lote del dataloader y cargarlo al dispositivo adecuado
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Limpiar gradientes existentes
        model.zero_grad()

        # Ejecutar el modelo para obtener logits
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Calcular la pérdida
        loss = loss_fn(outputs.logits, b_labels)
        total_loss += loss.item()

        # Retropropagación para calcular gradientes
        loss.backward()

        # Actualizar los parámetros del modelo
        optimizer.step()

    # Calcular la pérdida promedio sobre la época
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

Average training loss: 0.24340122679013246
Average training loss: 0.03561993770310845


In [50]:
#GUARDA EL MODELO DE ENTRENAMIENTO
model.save_pretrained(r"C:\Users\nobce\OneDrive\Documentos\PRUEBA CIENCITIFICO DATOS\2024_acl_diplomacy-master\modelo_de_entrenamiento_guardado")
tokenizer.save_pretrained(r"C:\Users\nobce\OneDrive\Documentos\PRUEBA CIENCITIFICO DATOS\2024_acl_diplomacy-master\modelo_de_entrenamiento_guardado")

('C:\\Users\\nobce\\OneDrive\\Documentos\\PRUEBA CIENCITIFICO DATOS\\2024_acl_diplomacy-master\\modelo_de_entrenamiento_guardado\\tokenizer_config.json',
 'C:\\Users\\nobce\\OneDrive\\Documentos\\PRUEBA CIENCITIFICO DATOS\\2024_acl_diplomacy-master\\modelo_de_entrenamiento_guardado\\special_tokens_map.json',
 'C:\\Users\\nobce\\OneDrive\\Documentos\\PRUEBA CIENCITIFICO DATOS\\2024_acl_diplomacy-master\\modelo_de_entrenamiento_guardado\\vocab.txt',
 'C:\\Users\\nobce\\OneDrive\\Documentos\\PRUEBA CIENCITIFICO DATOS\\2024_acl_diplomacy-master\\modelo_de_entrenamiento_guardado\\added_tokens.json')

In [51]:
# Tokenizamos los mensajes de la data de validación
validation_inputs, validation_masks = encode_messages(df_validation_copy['message'].tolist(), max_length=256)
# Se convierte las etiquetas en tensores
validation_labels = torch.tensor(df_validation_copy['sender_annotation'].values).long()  # Asegúrate de que 'label' es el nombre de tu columna de etiquetas

# Crea el TensorDataset
validation_dataset = TensorDataset(validation_inputs, validation_masks, validation_labels)
# Crea el DataLoader
batch_size = 16  # Puedes ajustar esto según las necesidades de tu modelo y hardware
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size)

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Función para evaluar el modelo
def evaluate_model(model, dataloader, device):
    model.eval()  # Poner el modelo en modo de evaluación
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            b_input_ids, b_input_mask, b_labels = batch
            b_input_ids = b_input_ids.to(device)
            b_input_mask = b_input_mask.to(device)
            b_labels = b_labels.to(device)

            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            predictions.append(logits)
            true_labels.append(label_ids)

    flat_predictions = np.concatenate(predictions, axis=0)
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_true_labels = np.concatenate(true_labels, axis=0)

    return flat_predictions, flat_true_labels

# Asumiendo que tienes un DataLoader para tus datos de validación/prueba
predictions, true_labels = evaluate_model(model, validation_dataloader, device)

print("Classification Report:")
print(classification_report(true_labels, predictions))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predictions))

print("Accuracy Score:")
print(accuracy_score(true_labels, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.16      0.12        56
           1       0.96      0.94      0.95      1360

    accuracy                           0.91      1416
   macro avg       0.53      0.55      0.54      1416
weighted avg       0.93      0.91      0.92      1416

Confusion Matrix:
[[   9   47]
 [  80 1280]]
Accuracy Score:
0.9103107344632768


In [54]:
#Segun nuestro modelo, el accuracy es de 0.93, lo cual es bastante bueno, pero no es suficiente para nuestro modelo
#lo cual es aconsajable ajustar learning rate o los epochs con mas tiempo 

#PRUEBA SOBRE EL CONJUNTO DE DATOS DE PRUEBA df_test_copy

# Tokenizar los textos de prueba
test_inputs, test_masks = encode_messages(df_test_copy['message'].tolist(), max_length=256)
# Convertir las etiquetas de prueba en tensores
test_labels = torch.tensor(df_test_copy['sender_annotation'].values).long()
# Crear el TensorDataset
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
# Crear el DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=16)  # Ajusta el batch_size según sea necesario


In [56]:
# Evaluar el modelo con los datos nuevos que son los datos de df_test_copy
test_predictions, test_true_labels = evaluate_model(model, test_dataloader, device)

# Imprimir el reporte de clasificación y la matriz de confusión
print("Classification Report:")
print(classification_report(test_true_labels, test_predictions))

print("Confusion Matrix:")
print(confusion_matrix(test_true_labels, test_predictions))

print("Accuracy Score:")
print(accuracy_score(test_true_labels, test_predictions))


Classification Report:
              precision    recall  f1-score   support

           0       0.15      0.12      0.13       240
           1       0.92      0.93      0.93      2501

    accuracy                           0.86      2741
   macro avg       0.53      0.53      0.53      2741
weighted avg       0.85      0.86      0.86      2741

Confusion Matrix:
[[  28  212]
 [ 165 2336]]
Accuracy Score:
0.8624589565851879
